In [ ]:
#Importing modules
import numpy as np
from pomdp_new import Pomdp
import math

# Input data

The input data for the damage model is introduced in the cell below.
In this case, the model corresponds to a crack propagation law, where a crack grows over time.

In [ ]:
config = {}

config["T"] = 30 # Finite horizon in years
config["ncycles"] = 1000000 # Number of load cycle experienced in one year
config["d0_mean"] = 1 # Initial crack size
config["dcrit"] = 20  # Critical crack size
config["S_mean"] = 70 # Stress range definition in MPa
config["S_std"] = 10 # Crack growth variable
config["m"] = 3.5 # Crack growth variable
config["lnC_mean"] = -35.2 # Crack growth variable
config["lnC_std"] = 0.5 # Crack growth variable

In [ ]:
config

# Simulations

An object is loaded taking into account the previously defined damage model.
This is done by calling the class "Pomdp": Pomdp(config)

In [ ]:
model_dr = Pomdp(config)

The crack size distribution is propagated over time in a Monte Carlo simuation here below.
This is done through the method crack_growth(n_samples), where the number of samples are specified.
The output consists of crack growth results for the horizon and samples.

In [ ]:
n_samples = 1e6
model_dr.crack_growth(n_samples)

In [ ]:
model_dr.dd.shape

# Transition model

The transition model is below constructed by calling the method transition_model().
Note that this follows a particular discretisation scheme.

The output provides: 
* Inital damage probabilities: model_dr.b0
* Transition probabilities for a do-nothing action: model_dr.T0
* Transition probabilities for a repair action: model_dr.Tr

In [ ]:
model_dr.transition_model()

In [ ]:
model_dr.b0.shape

In [ ]:
model_dr.T0.shape

In [ ]:
model_dr.Tr.shape

# Inspection model

The inspection model is below constructed by calling the method observation_model(). 
Note that a specific probability of detection curve is here implemented.

The output provides: 
* Inspection model: model_dr.O

In [ ]:
model_dr.O

# Storing pomdp model

The transition and inspection models can be stored at the user's discretion and it is case dependent. 
The environment dynamics needs three main components:

* Initial probabilities: belief0  
* Transition probabilities: P
* Inspection model (or/and sensor model): O

In [ ]:
# Initial probabilities
belief0 = np.zeros((1, 10, 30, 1)) # ((empty, n_comp, damage, empty))
belief0[0, :, :, 0] = model_dr.b0

In [ ]:
# Transition model
P = np.zeros((3, 10, 31, 30, 30)) # ((action, n_comp, d_rate, damage, damage))
P[0, :, :, :, :] = model_dr.T0 # Do-nothing action
P[1, :, :, :, :] = model_dr.T0 # Inspection action
P[2, :, :, :, :] = model_dr.Tr # Repair action

In [ ]:
# Inspection model
O = np.zeros((3, 10, 30, 2)) # ((action, n_comp, damage, inspect_outcome))
O[0, :, :, 0] = np.ones(30) # Do-nothing action
O[1, :, :, :] = model_dr.O # Inspection action
O[2, :, :, 0] = np.ones(30) # Repair action

In [ ]:
# Store it (for instance as a numpy file)
np.savez('pomdp_model.npz', belief0=belief0, P=P, O=O)